In [ ]:
import matplotlib.pyplot as plt
from ekpy import control
from ekpy.control.instruments import keysight81150a, keysightdsox3024a
from PVloop import FEHysteresis
import numpy as np

In [ ]:
rm = control.ResourceManager()
rm.list_resources()

In [ ]:
scope = rm.open_resource('USB0::0x0957::0x17A6::MY63080078::INSTR')
wavegen = rm.open_resource('GPIB0::8::INSTR')

In [ ]:
keysightdsox3024a.idn(scope)

In [ ]:
keysight81150a.idn(wavegen)

In [ ]:
exp_fe = FEHysteresis(wavegen,scope)

# specify run function (if desired)
# options are: single_pulse_run_function, two_pulse_run_function, preset_run_function


# config data path
exp_fe.config_path('./test')

In [ ]:
initial_delays = ['5e-5', '6e-5']
scan_params = {
    'initial_delay':initial_delays

}

run_function_args = {
    'wavegen':wavegen,
    'scope':scope,
    'initial_delay':'5e-5',
    'pulse_delay':'5e-6',
    'freq': '10',
    'v_end':'2',
    'capacitor_area':'4e-8',
    'thickness':'10e-9',
    'permittivity':'30',
	'amplification':'8',
}

order = ['initial_delay',]

In [ ]:
exp_fe.n_param_scan(scan_params, run_function_args, order, ntrials=1, plot=False)